In [17]:
# сменим рабочую директорию на корневую
%cd ..
!pwd

/home/jupyter/work/resources/SR-Gaming-Bench


In [14]:
# submodules нужно скачать вручную, иначе падает с ошибкой
!cd submodules/ && \
    git clone https://github.com/Pe4enIks/BasicSR.git && \
    mv BasicSR basicsr && \
    git clone https://github.com/Pe4enIks/Real-ESRGAN.git && \
    mv Real-ESRGAN real-esrgan && \
    git clone https://github.com/Pe4enIks/ResShift && \
    mv ResShift resshift

Cloning into 'BasicSR'...
Cloning into 'Real-ESRGAN'...
Cloning into 'ResShift'...
Updating files: 100% (312/312), done.


In [ ]:
# зависимости берем из requirements.txt, потому что через poetry нет возможности
%pip install -r requirements.txt

In [38]:
# dvc pull не сработает, поэтому создадим нужные папки и выкачаем веса с yandex disk
!mkdir -p dvc_data/weights/real-esrgan

In [39]:
# скрипт скачивания весов с yandex disk
from urllib.parse import urlencode

import requests


def download_file(pk, local_name):
    base_url = "https://cloud-api.yandex.net/v1/disk/public/resources/download?"

    final_url = base_url + urlencode(dict(public_key=pk))
    response = requests.get(final_url)
    download_url = response.json()["href"]

    download_response = requests.get(download_url)
    with open(local_name, "wb") as f:
        f.write(download_response.content)
        print(f"File {local_name} downloaded")


files = {
    "dvc_data/weights/real-esrgan/RealESRGAN_x4plus_netD.pth": "5KqCavy3QNqK9w",
    "dvc_data/weights/real-esrgan/RealESRNet_x4plus.pth": "IGcPWnlJoalrKw",
}


print(f"Will be download {len(files)} files")
for filename, file_id in files.items():
    link = f"https://disk.yandex.ru/d/{file_id}"
    print(f"{filename} downloading in progress")
    download_file(link, filename)
    print(f"{filename} downloaded")

Will be download 2 files
dvc_data/weights/real-esrgan/RealESRGAN_x4plus_netD.pth downloading in progress
File dvc_data/weights/real-esrgan/RealESRGAN_x4plus_netD.pth downloaded
dvc_data/weights/real-esrgan/RealESRGAN_x4plus_netD.pth downloaded
dvc_data/weights/real-esrgan/RealESRNet_x4plus.pth downloading in progress
File dvc_data/weights/real-esrgan/RealESRNet_x4plus.pth downloaded
dvc_data/weights/real-esrgan/RealESRNet_x4plus.pth downloaded


In [61]:
# для возможности получить доступ к приватному репозиторию huggingface
import huggingface_hub as hf_hub

hf_hub.login()

In [ ]:
# скачаем необходимые данные через huggingface api
!mkdir -p "datasets/tmp"

hf_hub.snapshot_download(
    repo_id="epishchik/super-resolution-games",
    repo_type="dataset",
    local_dir="datasets/tmp",
    cache_dir="hf_cache",
    local_dir_use_symlinks=True,
    allow_patterns=[
        "data/GameEngineData/*270p.tar.gz",
        "data/GameEngineData/*1080p.tar.gz",
    ],
)

In [ ]:
# скрипт формирования датасета в нужном формате
!cd utils && bash create_local_dataset.sh \
    /home/jupyter/work/resources/SR-Gaming-Bench/datasets \
    GameEngineData \
    270p \
    1080p \
    false